In [14]:
!pip install sentence-transformers
!pip install langchain
!pip install PyPDF
!pip install transformers
!pip install faiss-cpu
!pip install langchain openai
!pip install pypdf
!pip install tiktoken

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
import numpy as np

# PDF to Vector-Store(Ingestion)

In [16]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import pickle
from langchain.document_loaders import PyPDFLoader

# Load the document
loader = PyPDFLoader("/content/Casewise Internship.pdf")
raw_documents = loader.load()

# Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)

chunks = text_splitter.split_documents(raw_documents)
# Creating Embeddings
embedding_model_id = "BAAI/bge-small-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)
# Saving Embeddings in VectorStore
embeddings_db = FAISS.from_documents(chunks, embeddings)
embeddings_db.save_local("faiss_index")



# RAG Implementation (Generation)

In [17]:
import openai

In [18]:
# use your own openai api key
openai.api_key = "sk-"

In [19]:
system_template = """Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}

Begin!
----------------
Question: {question}
Helpful Answer:"""

In [20]:

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [21]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613",openai_api_key = openai.api_key)


In [22]:
question = input("Write your question here?")


Write your question here?casewise is what kind of startup?


In [23]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=embeddings_db.as_retriever(),
    chain_type_kwargs = {"prompt": prompt})
result = qa_chain({"query": question})


In [24]:
print(result)

{'query': 'casewise is what kind of startup?', 'result': 'Casewise is a legal tech startup.'}
